In [64]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *


In [65]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/YTS"

## Dataset

In [66]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [67]:
bs=52 # 52 - Jeremey, 20 - default

In [68]:
bptt= 70 #70 - Jeremey, 35 - default

In [69]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [70]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
train_tokens.shape, valid_tokens.shape

((52, 927), (52, 101))

In [71]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [72]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [73]:
x.shape,y.shape

((52, 70), (52, 70))

In [74]:
x,y

(array([[  5,   6,   7,   4, ...,  21,  66,  86, 750],
        [186,   3, 272,  22, ...,  26,   0,   5,   6],
        [  0,  36,   2,   0, ...,  25,  55, 923,   0],
        [  0,   0,   0,   0, ...,   0,   5,   6,   7],
        ...,
        [ 59,   8,  10,  79, ..., 162,  20,  19,  12],
        [ 11,  15, 153,  33, ...,   5,   6,   7,   4],
        [ 11,  22,  29, 155, ..., 568,  27, 283,  18],
        [ 27, 100, 515,  46, ..., 130,  94,   2,  40]]),
 array([[  6,   7,   4,   2, ...,  66,  86, 750,   9],
        [  3, 272,  22,   3, ...,   0,   5,   6,   7],
        [ 36,   2,   0,   0, ...,  55, 923,   0,   5],
        [  0,   0,   0,   0, ...,   5,   6,   7,   4],
        ...,
        [  8,  10,  79,  52, ...,  20,  19,  12,   3],
        [ 15, 153,  33, 145, ...,   6,   7,   4,   2],
        [ 22,  29, 155,  65, ...,  27, 283,  18, 250],
        [100, 515,  46,  27, ...,  94,   2,  40, 263]]))

In [75]:
x.shape,y.shape

((52, 70), (52, 70))

In [76]:
train_tokens.shape

(52, 927)

## pre-trained AWD-LSTM

In [77]:
PATH_AWD_LSTM='/home/kirana/Documents/phd/data/pre-trained/awd_lstm/'

In [78]:
ls {PATH_AWD_LSTM}

bwd_wt103_enc.h5  fwd_wt103_enc.h5  itos_wt103.pkl  wt103_60002/
bwd_wt103.h5      fwd_wt103.h5      wt103_238642/   wt103_tiny.tgz


In [79]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103.h5', map_location=lambda storage, loc: storage)

In [80]:
wgts

OrderedDict([('0.encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('0.encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                

In [81]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103_enc.h5', map_location=lambda storage, loc: storage)

In [82]:
wgts

OrderedDict([('encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                    

In [83]:
itos2=pickle.load(open(f'{PATH_AWD_LSTM}/itos_wt103.pkl','rb'))
stoi2 = collections.defaultdict(lambda: -1, { v: k for k, v in enumerate(itos2) })

In [84]:
wgts['encoder.weight'].shape

torch.Size([238462, 400])

In [85]:
enc_wgts = wgts['encoder.weight'].numpy() # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [86]:
enc_wgts.shape, len(itos)

((238462, 400), 1157)

In [87]:
n_emb=enc_wgts.shape[1]

In [88]:
new_w = np.zeros((len(itos),n_emb), dtype=np.float32) # shape: (60002, 400)

for i, w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r >= 0 else row_m

In [89]:
len(set(itos).difference(set(itos2)))

175

## Model Architecture

In [90]:
from torch import nn
import torch
from adaptive import *
from torch.autograd import Variable

In [91]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400 #650
n_layers=2
dropout=0.5
wd=1e-7

In [92]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [93]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [94]:
device="cuda:1"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


initializing


In [95]:
new_w.shape

(1157, 400)

In [96]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([1157, 400]), torch.Size([1157, 400]))

In [97]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([1157, 400]), 400, 1157)

### Test if model forward works

In [98]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([1157, 400]), torch.Size([1157, 400]))

In [99]:
x.shape

(52, 70)

In [100]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [101]:

model=model.to(device)

## Learner

In [102]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1

    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.gen_hidden()
        #for k,i in enumerate(range(0,n_batch,self.bptt)):
        n_batch=iterator.shape[1]
        while i<n_batch-bptt:
            if mode_train:
                cust_bptt=self.bptt if np.random.random() < 0.95 else self.bptt//np.random.randint (2,4)
            else:
                cust_bptt=bptt
            seq_len=min(cust_bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
            k=k+1
            i=i+seq_len
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [103]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [104]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

14

In [105]:
model.freeze_embedding()

In [106]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [107]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [108]:
model.n_inp

1157

In [109]:
learner.run_epochs(train_tokens,valid_tokens,100)

Epoch:0 Loss:6.263374475332407
Epoch:0 Loss:6.263374475332407 Accuracy:0.06565934262023522 Loss:5.697565078735352 Accuracy:0.1329670399427414
Epoch:1 Loss:5.719012223757231
Epoch:1 Loss:5.719012223757231 Accuracy:0.12466188233632308 Loss:5.336146354675293 Accuracy:0.17142857611179352
Epoch:2 Loss:5.43531689277062
Epoch:2 Loss:5.43531689277062 Accuracy:0.1581572385934683 Loss:5.096234321594238 Accuracy:0.19203297793865204
Epoch:3 Loss:5.242917390970083
Epoch:3 Loss:5.242917390970083 Accuracy:0.17709490198355454 Loss:4.90220832824707 Accuracy:0.21291209757328033
Epoch:4 Loss:5.111061793107253
Epoch:4 Loss:5.111061793107253 Accuracy:0.19025782438424918 Loss:4.76827335357666 Accuracy:0.2260989099740982
Epoch:5 Loss:4.993988110468938
Epoch:5 Loss:4.993988110468938 Accuracy:0.20200761694174546 Loss:4.640859603881836 Accuracy:0.23571430146694183
Epoch:6 Loss:4.89334381543673
Epoch:6 Loss:4.89334381543673 Accuracy:0.21107355447915885 Loss:4.542698383331299 Accuracy:0.24725276231765747
Epoch:7 

Epoch:57 Loss:2.6595315749828634
Epoch:57 Loss:2.6595315749828634 Accuracy:0.46570897560853225 Loss:1.9695342779159546 Accuracy:0.6140109896659851
Epoch:58 Loss:2.6587691123668966
Epoch:58 Loss:2.6587691123668966 Accuracy:0.46804735981501067 Loss:1.9412063360214233 Accuracy:0.612912118434906
Epoch:59 Loss:2.616838812828064
Epoch:59 Loss:2.616838812828064 Accuracy:0.4736519881657192 Loss:1.9311718940734863 Accuracy:0.6167582869529724
Epoch:60 Loss:2.5964413422804613
Epoch:60 Loss:2.5964413422804613 Accuracy:0.4762257177096147 Loss:1.8960481882095337 Accuracy:0.6263736486434937
Epoch:61 Loss:2.5996432644980296
Epoch:61 Loss:2.5996432644980296 Accuracy:0.4791840272290366 Loss:1.8853416442871094 Accuracy:0.6258242130279541
Epoch:62 Loss:2.568535327911377
Epoch:62 Loss:2.568535327911377 Accuracy:0.4809805773771726 Loss:1.8517916202545166 Accuracy:0.634615421295166
Epoch:63 Loss:2.534862756729126
Epoch:63 Loss:2.534862756729126 Accuracy:0.4863482851248521 Loss:1.8238099813461304 Accuracy:0.6

In [110]:
learner.run_epochs(train_tokens,valid_tokens,60)

Epoch:0 Loss:1.890881428351769
Epoch:0 Loss:1.890881428351769 Accuracy:0.5864328237680289 Loss:1.2536641359329224 Accuracy:0.7467033267021179
Epoch:1 Loss:1.8643977091862605
Epoch:1 Loss:1.8643977091862605 Accuracy:0.588669226719783 Loss:1.2419167757034302 Accuracy:0.7513736486434937
Epoch:2 Loss:1.862069927729093
Epoch:2 Loss:1.862069927729093 Accuracy:0.5904057805354779 Loss:1.2317347526550293 Accuracy:0.7524725794792175
Epoch:3 Loss:1.8186275042020357
Epoch:3 Loss:1.8186275042020357 Accuracy:0.6005283502432016 Loss:1.2225782871246338 Accuracy:0.7543956637382507
Epoch:4 Loss:1.8147938251495361
Epoch:4 Loss:1.8147938251495361 Accuracy:0.5990279179352981 Loss:1.2105119228363037 Accuracy:0.7560439705848694
Epoch:5 Loss:1.8257078102656774
Epoch:5 Loss:1.8257078102656774 Accuracy:0.5959201029368809 Loss:1.1980080604553223 Accuracy:0.7568681836128235
Epoch:6 Loss:1.8033296878521259
Epoch:6 Loss:1.8033296878521259 Accuracy:0.6006762752166162 Loss:1.192304253578186 Accuracy:0.758791267871856

Epoch:57 Loss:1.3989505859521718
Epoch:57 Loss:1.3989505859521718 Accuracy:0.6621302045308627 Loss:0.939484715461731 Accuracy:0.7865384817123413
Epoch:58 Loss:1.39860449387477
Epoch:58 Loss:1.39860449387477 Accuracy:0.6615384862973139 Loss:0.9373356699943542 Accuracy:0.7868131995201111
Epoch:59 Loss:1.3841655804560735
Epoch:59 Loss:1.3841655804560735 Accuracy:0.6636517689778254 Loss:0.9309297800064087 Accuracy:0.7881868481636047


In [111]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict')

In [112]:
model.unfreeze_embedding()

In [113]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [114]:
learner.run_epochs(train_tokens,valid_tokens,100)

Epoch:0 Loss:1.3092224322832549
Epoch:0 Loss:1.3092224322832549 Accuracy:0.6738377305177542 Loss:0.8163505792617798 Accuracy:0.8186813592910767
Epoch:1 Loss:1.1967030671926646
Epoch:1 Loss:1.1967030671926646 Accuracy:0.7120456878955548 Loss:0.7562476992607117 Accuracy:0.8439561128616333
Epoch:2 Loss:1.1329299440750709
Epoch:2 Loss:1.1329299440750709 Accuracy:0.7325857510933509 Loss:0.7096996903419495 Accuracy:0.8565934300422668
Epoch:3 Loss:1.1043552527060876
Epoch:3 Loss:1.1043552527060876 Accuracy:0.7453931019856379 Loss:0.6627073287963867 Accuracy:0.8700549602508545
Epoch:4 Loss:1.0509100877321684
Epoch:4 Loss:1.0509100877321684 Accuracy:0.758495385830219 Loss:0.6220210790634155 Accuracy:0.8750000596046448
Epoch:5 Loss:1.0063160061836243
Epoch:5 Loss:1.0063160061836243 Accuracy:0.7673288583755493 Loss:0.5852211713790894 Accuracy:0.8840659856796265
Epoch:6 Loss:0.9578913817038903
Epoch:6 Loss:0.9578913817038903 Accuracy:0.7757819478328412 Loss:0.5505117177963257 Accuracy:0.8917582631

Epoch:57 Loss:0.4029394250649672
Epoch:57 Loss:0.4029394250649672 Accuracy:0.8898554077515235 Loss:0.14322823286056519 Accuracy:0.966208815574646
Epoch:58 Loss:0.4101918133405539
Epoch:58 Loss:0.4101918133405539 Accuracy:0.8878064568226154 Loss:0.13585084676742554 Accuracy:0.9686813354492188
Epoch:59 Loss:0.4111857781043419
Epoch:59 Loss:0.4111857781043419 Accuracy:0.8881023251093351 Loss:0.13609245419502258 Accuracy:0.9678571820259094
Epoch:60 Loss:0.3981272807488075
Epoch:60 Loss:0.3981272807488075 Accuracy:0.8920688399901757 Loss:0.13106219470500946 Accuracy:0.9708791375160217
Epoch:61 Loss:0.39515947149350095
Epoch:61 Loss:0.39515947149350095 Accuracy:0.8924979200729957 Loss:0.13129663467407227 Accuracy:0.9703297019004822
Epoch:62 Loss:0.39794933337431687
Epoch:62 Loss:0.39794933337431687 Accuracy:0.8911242989393381 Loss:0.12703180313110352 Accuracy:0.971428632736206
Epoch:63 Loss:0.3880743602911631
Epoch:63 Loss:0.3880743602911631 Accuracy:0.8934201598167419 Loss:0.127283886075019

In [118]:
np.exp(0.2737),np.exp(0.071)

(1.314820296976801, 1.0735812258683575)

In [115]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict_unfreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict_unfreeze')

In [116]:
torch.save (model,f'{DATAPATH}/inter/varybptt_model_awd_lstm')
torch.save (optimizer,f'{DATAPATH}/inter/varybptt_optimizer_awd_lstm')
torch.save (learner,f'{DATAPATH}/inter/varybptt_learner_awd_lstm')

In [117]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
import pickle
pickle.dump(pretrained_lm_weights,open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights','wb'))